In [1]:
from TM1py.Services import TM1Service
from TM1py.Utils import Utils
import pandas as pd
from sklearn.linear_model import LinearRegression
import time
import matplotlib.pyplot as plt
import re
from typing import List
import asyncio
import nest_asyncio
nest_asyncio.apply()
from concurrent.futures import ThreadPoolExecutor

### 获取Training 实际数部分

In [2]:
zl_report_all=pd.read_csv(r'C:\TM1py\zl_amazon_payments_data_report.csv',
                   dtype={'SA Year':'str','Order Year':'str','COA Company':'str','Platform':'str'})

### 特征列与预测销量的回归

In [3]:
zl_report_all

,Scenario,SA Year,SA Month,Version,COA Company,Currency,Country,Product,Platform,ZL Amazon Management Report Item,Data Source Payments Data Report,M ZL Amazon Payments Data Report,Value
0,ACT,2018,M01,WIP,002,USD,US,LSKU1787,07,AMZ SP广告费分摊,Base,Amount,278.9424
1,ACT,2018,M01,WIP,002,USD,US,LSKU1787,07,AMZ SD广告费分摊,Base,Amount,43.1232
2,ACT,2018,M01,WIP,002,USD,US,LSKU1284,07,AMZ SP广告费分摊,Base,Amount,161.3496
3,ACT,2018,M01,WIP,002,USD,US,LSKU1284,07,AMZ SD广告费分摊,Base,Amount,0.2784
4,ACT,2018,M01,WIP,002,USD,US,LSKU1931,07,AMZ SP广告费分摊,Base,Amount,79.6800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6706997,Forecast,2021,M12,WIP,002,No Currency,SE,LSKU1179,05,销售数量(不含退货),Base,Amount,1.0000
6706998,Forecast,2021,M12,WIP,002,No Currency,SE,LSKU1904,05,退货数量,Base,Amount,-1.0000
6706999,Forecast,2021,M12,WIP,002,No Currency,SE,LSKU1303,05,销售数量(不含退货),Base,Amount,1.0000
6707000,Forecast,2021,M12,WIP,002,No Currency,TR,LSKU1929,06,销售数量(不含退货),Base,Amount,1.0000


In [5]:
dict_linearRegress=dict()

async def Linear_async(df:'pd.DataFrame',Year:List):
    loop = asyncio.get_event_loop()
    
    with ThreadPoolExecutor(8) as executor:
        #futures = [loop.rudata:'pd.DataFrame',n_in_executor(executor, transfer, tm1_src, tm1_tgr,cube_src, cube_tgr,month_leaf) for month_leaf in leaves]
        futures=[]
        for year in Year:
            
            f=loop.run_in_executor(executor, linear_model_ACT,df,year)
            futures.append(f)
        for future in futures:
            try:
                await future
            except:
                print(future)
                continue      

def linear_model_ACT(df:'pd.DataFrame',year:str):
    def transfer_c(df:'pd.DataFrame',year:str):

        df_filter=df[(df['Scenario']=='ACT')&(df['Currency']=='USD') &(df['SA Year']==year) 
                        &((df['ZL Amazon Management Report Item']=='销售数量(不含退货)')|(df['ZL Amazon Management Report Item']=='AMZ SP广告费分摊')
                        |(df['ZL Amazon Management Report Item']=='AMZ SD广告费分摊')
                        |(df['ZL Amazon Management Report Item']=='AMZ SBV广告费分摊')
                        |(df['ZL Amazon Management Report Item']=='AMZ SB广告费分摊')
                        |(df['ZL Amazon Management Report Item']=='AMZ DSP广告费分摊'))]
        df_pivot=df_filter.pivot(index=['Scenario','SA Year','SA Month','Version','COA Company','Currency','Country','Product','Platform','Data Source Payments Data Report','M ZL Amazon Payments Data Report']
                            ,columns='ZL Amazon Management Report Item',values='Value').reset_index().reset_index(drop=True)
        df_pivot.fillna(0,inplace=True)
        return  df_pivot
    df=transfer_c(df,year)
    #聚合AD
    df['全部广告费']=df.iloc[:,11]+df.iloc[:,12]+df.iloc[:,13]+df.iloc[:,14]+df.iloc[:,15]
    #自变量定义
    Z=df[['全部广告费']]
    #回归
    lr=LinearRegression()
    lr.fit(Z,df['销售数量(不含退货)'])
    print('R Square:',lr.score(Z,df['销售数量(不含退货)']))
    print('intercept:',lr.intercept_)
    print('coef',lr.coef_)
    dict_linearRegress[year]=lr
    return lr 

In [6]:
year_list=['2021']
asyncio.run(Linear_async(zl_report_all,year_list))

R Square: 0.2290883212564786
intercept: 36.7452214028892
coef [0.11745228]


In [21]:
dict_linearRegress['2021'].coef_

array([0.11745228])

#### 连接Tm1

In [22]:
tm1 = TM1Service(address="192.168.0.176", port=30015, ssl=False, user="neil", password="123")

#### 处理要预测填报的数据
##### 处理某维度中包含的特征列，将之透视变成真正的平铺的列的样式，方便回归

In [34]:
#处理目标cube的数据
def transfer_fcst(tm1: TM1Service,year:str,year_fcst:str,month:str):
        mdx='''
        SELECT 
          NON EMPTY 
           {Tm1filterbylevel({[ZL AMZ Fee Item].[ZL AMZ Fee Item].Members} ,0)}
            ON COLUMNS , 
          NON EMPTY 
           {Tm1filterbylevel({[Product].[Product].Members},0)}
           *{[SA Month].[SA Month].['''+month+''']}
           *{Tm1filterbylevel( {[Platform].[Platform].Members},0)}
           *{Tm1filterbylevel( {[Fee Allocation Method].[Fee Allocation Method].Members} ,0)}  
            ON ROWS 
        FROM [Sys Definition ZL AMZ Fee STG] 
        WHERE 
          (
           [Scenario].[Scenario].[FCST M01 CY2022],
           [SA Year].[SA Year].['''+year_fcst+'''],
           [COA Company].[COA Company].[002],
           [Version].[Version].[WIP],
           [Data Source ZL AMZ Fee STG].[Data Source ZL AMZ Fee STG].[Base],
           [M Sys Definition ZL AMZ Fee STG].[M Sys Definition ZL AMZ Fee STG].[Amount],
           [Currency].[Currency].[USD],
           [Country].[Country].[US]
          )
             '''
        #元组字典
        cells=tm1.cubes.cells.execute_mdx(mdx,skip_rule_derived_cells=True,skip_zeros=True,skip_consolidated_cells=True)
        df_nlevel = Utils.build_pandas_dataframe_from_cellset(cells)
        df_nlevel=df_nlevel.reset_index()
        df_pivot_n=df_nlevel.pivot(index=['Scenario','SA Year','SA Month','Version','COA Company','Currency','Country','Platform','Product','Fee Allocation Method'
                              ,'Data Source ZL AMZ Fee STG','M Sys Definition ZL AMZ Fee STG'],columns='ZL AMZ Fee Item'
                      ,values='Values').reset_index().reset_index(drop=True)
        #清洗NA
        df_pivot_n.fillna(0,inplace=True)
        #插值法
#         from scipy.interpolate import lagrange
#         def lag_inter(s):
#             for n in range(0,len(s)):
#                 if s.isnull()[n]:
#                     #y=s.iloc[(n-150):(n+150+150),:]
#                     #y.dropna(inplace=True)
#                     #v=lagrange(y.index,list(y))(n)
#                     s.iloc[n,:]=0
#         lag_inter(df_pivot_n['AMZ DSP广告费分摊'])
#         lag_inter(df_pivot_n['AMZ SB广告费分摊'])
#         lag_inter(df_pivot_n['AMZ SBV广告费分摊'])
#         lag_inter(df_pivot_n['AMZ SD广告费分摊'])
#         lag_inter(df_pivot_n['AMZ SP广告费分摊'])
#         df_pivot_n.dropna(inplace=True)
        #聚合广告费
        df_pivot_n['全部广告费']=df_pivot_n.iloc[:,12]+df_pivot_n.iloc[:,13]+df_pivot_n.iloc[:,14]+df_pivot_n.iloc[:,15]+df_pivot_n.iloc[:,16]
        return df_pivot_n

### 预测部分

#### 将数据处理成Tm1可识别的逆透视样式

In [45]:
def linear_model_FCST(df:'pd.DataFrame',lr:'LinearRegress list',year:str,year_fcst:str,month:str):
    #回归预测
    Z_tgr=df[['全部广告费']]
    y_predict=lr[year].predict(Z_tgr)
    df['Sales Units']=y_predict
    df.drop(['全部广告费'],axis=1,inplace=True)
    df_ready=df.melt(id_vars=['Scenario','SA Year','SA Month','Version','COA Company','Currency','Country','Platform','Product','Fee Allocation Method'
                        ,'Data Source ZL AMZ Fee STG','M Sys Definition ZL AMZ Fee STG'],var_name=['ZL AMZ Fee Item'],value_name='Values')
    #获取待回写的Dataframe   d
    return df_ready   

### Tm1中写入预测数据

In [43]:
async def fcst_async(tm1:TM1Service,lr:'LinearRegress list',year:str,year_fcst:str,months:List,cube_tgr:str):
    loop = asyncio.get_event_loop()
    
    with ThreadPoolExecutor(8) as executor:
        #futures = [loop.rudata:'pd.DataFrame',n_in_executor(executor, transfer, tm1_src, tm1_tgr,cube_src, cube_tgr,month_leaf) for month_leaf in leaves]
        futures=[]
        for month in months:
            
            f=loop.run_in_executor(executor, linear_transfer,tm1,lr,year,year_fcst,month,cube_tgr)
            futures.append(f)
        for future in futures:
            try:
                await future
            except:
                print(future)
                continue  



def linear_transfer(tm1:TM1Service,lr:'LinearRegress list',year:str,year_fcst:str,month:str,cube_tgr:str):
    
    df=transfer_fcst(tm1,year,year_fcst,month)
    
    df=linear_model_FCST(df,lr,year,year_fcst,month)
    df['Business Unit']='hangzhou'
    df['Customer']='No Amazon Customer'
    df['M ZL Product Sales Plan']='Sales Units'
    df['Data Source ZL Product Sales Plan']='Linear FCST'
    df['Channel']='Amazon'
    df=df[df['ZL AMZ Fee Item']=='Sales Units']
    df.drop(['Platform','Fee Allocation Method','Data Source ZL AMZ Fee STG','M Sys Definition ZL AMZ Fee STG','ZL AMZ Fee Item'],axis=1,inplace=True)
    df.rename(columns={'Country':'Region','ZL AMZ Fee Item':'M ZL Product Sales Plan'},inplace=True)
    col_name=['Scenario','SA Year','SA Month','Version','COA Company','Business Unit','Product','Currency','Region','Channel','Customer','Data Source ZL Product Sales Plan','M ZL Product Sales Plan','Values']
    df_final=df.reindex(columns=col_name)
    #clear
    mdx_tgr='''
        SELECT 
           {Tm1filterbylevel({[Region].[Region].Members},0)}
           *{[SA Month].[SA Month].['''+month+''']}
          ON COLUMNS , 
           {Tm1filterbylevel( {[Product].[Product].Members},0)}
           *{Tm1filterbylevel( {[Customer].[Customer].Members},0)}
           ON ROWS 
        FROM [ZL Product Sales Plan] 
        WHERE 
          (
           [Scenario].[Scenario].[FCST M01 CY2022],
           [SA Year].[SA Year].['''+year_fcst+'''],
           [Version].[Version].[WIP],
           [COA Company].[COA Company].[002],
           [Data Source ZL Product Sales Plan].[Data Source ZL Product Sales Plan].[Linear FCST],
           [Currency].[Currency].[USD],
           [M ZL Product Sales Plan].[M ZL Product Sales Plan].[Sales Units],
           [Business Unit].[Business Unit].[hangzhou],
           [Channel].[Channel].[Amazon]
  )
             '''
    tm1.cells.clear_with_mdx(cube_tgr,mdx_tgr)
    #Writing to tgr Cube
    tm1.cells.write_dataframe(
        cube_name=cube_tgr,
        data=df_final,
        dimensions=['Scenario','SA Year','SA Month','Version','COA Company','Business Unit','Product','Currency'
                    ,'Region','Channel','Customer','Data Source ZL Product Sales Plan','M ZL Product Sales Plan'],
        increment=True,
        deactivate_transaction_log=True,
        reactivate_transaction_log=True,
        sandbox_name=None,
        use_ti=True,
        skip_non_updateable=False,
        use_changeset=False)

In [44]:
months_list=['M01','M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12']
asyncio.run(fcst_async(tm1,dict_linearRegress,'2021','2022',months_list,'ZL Product Sales Plan'))